In [1]:
# 1. Install dependencies
!pip install datasets transformers tqdm


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
# 2. Imports
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from tqdm import tqdm
import json
import random

In [3]:
# 4. Load TinyLLaMA model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
# 5. Load 5000 samples from Open-Instruct
open_instruct = load_dataset("hakurei/open-instruct-v1")["train"].select(range(5000))

In [5]:
# 6. Define perturbation function
def get_perturbed_instruction(instruction):
    prompt = f"Rewrite the following prompt to confuse a language model but keep its meaning the same:\nPrompt: {instruction}\nAdversarial Version:"
    outputs = generator(prompt, max_new_tokens=60, do_sample=True, top_k=50, temperature=0.9)
    perturbed = outputs[0]["generated_text"].split("Adversarial Version:")[-1].strip()
    return perturbed

In [6]:
# 7. Create adversarial dataset with progress bar
adversarial_dataset = []
for sample in tqdm(open_instruct, desc="Generating adversarial samples"):
    try:
        perturbed_instruction = get_perturbed_instruction(sample["instruction"])
        adversarial_dataset.append({
            "instruction": perturbed_instruction,
            "input": sample["input"],
            "output": sample["output"]
        })
    except Exception as e:
        print(f"Skipping due to error: {e}")
        continue
 
# 8. Save to disk using save_to_disk() instead of Drive
output_path = "./adversarial5000"
adversarial_dataset.save_to_disk(output_path)
 
print(f"\n✅ Saved {len(adversarial_dataset)} adversarial samples to {output_path}")

Generating adversarial samples: 100%|██████████| 5000/5000 [45:38<00:00,  1.83it/s]


AttributeError: 'list' object has no attribute 'save_to_disk'

In [9]:
from datasets import Dataset

# Assuming adversarial_dataset is a list of dicts like: [{"text": ..., "label": ...}, ...]
adversarial_dataset = Dataset.from_list(adversarial_dataset)

# Now save to disk
output_path = "./adversarial5000"
adversarial_dataset.save_to_disk(output_path)

print(f"\n✅ Saved {len(adversarial_dataset)} adversarial samples to {output_path}")


Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]


✅ Saved 5000 adversarial samples to ./adversarial5000
